In [1]:
import pandas as pd
import numpy as np
import datetime
#import re

In [2]:
df = pd.read_csv('simulatedData.csv')

In [3]:
df

,userId,sessionId,eventTime,pageName,device
0,user_144,u144s0,2025-01-01 01:58:46,start,IOS
1,user_144,u144s0,2025-01-01 02:02:29,info,IOS
2,user_144,u144s0,2025-01-01 02:07:42,start,Android
3,user_144,u144s0,2025-01-01 02:17:20,step1,Web
4,user_144,u144s0,2025-01-01 02:27:01,help1,Web
...,...,...,...,...,...
7402,user_895,u895s1,2025-01-24 01:23:34,start,IOS
7403,user_410,u410s3,2025-01-27 04:17:22,help2,Android
7404,user_762,u762s1,2025-01-08 02:43:19,question,IOS
7405,user_891,u891s1,2025-01-06 05:02:41,start,Web


In [4]:
df['userId']=df['userId'].astype(str)
df['sessionId']=df['sessionId'].astype(str)
df['eventTime']=pd.to_datetime(df['eventTime'])

In [5]:
print('原始行数：', len(df))
print('完全重复行数：', df.duplicated().sum())
print('pageName缺失数量：', df['pageName'].isna().sum())
print('最晚时间：',df['eventTime'].max())
print('最早时间：',df['eventTime'].min())
print('不同页面名称计数：',df['pageName'].value_counts())

原始行数： 7407
完全重复行数： 30
pageName缺失数量： 74
最晚时间： 2025-02-10 14:38:19
最早时间： 1970-01-01 00:00:00
不同页面名称计数： start       2171
exit        1451
step1        915
step2        863
info         656
help2        444
checkout     260
question     219
help1        172
error         77
success       31
Start         31
Exit          15
Step1          8
Step2          6
Help2          6
Checkout       4
Info           3
Help1          1
Name: pageName, dtype: int64


In [6]:
n=len(df)
df=df.drop_duplicates()
print('已去掉',n-len(df),'重复行')

已去掉 30 重复行


In [7]:
judge=df['eventTime']<pd.to_datetime('2020-01-01')
df=df.loc[~judge].copy().reset_index(drop=True)
print('已去掉',judge.sum(),'时间异常行')

已去掉 7 时间异常行


In [8]:
df['pageName']=df['pageName'].astype('string').str.strip()
df['pageName']=df['pageName'].str.lower()

In [9]:
df['pageName'].value_counts()

start       2189
exit        1462
step1        918
step2        867
info         652
help2        449
checkout     263
question     217
help1        172
error         76
success       31
Name: pageName, dtype: Int64

In [10]:
df=df[df['pageName'].notna()].copy().reset_index(drop=True)

In [11]:
df=df.drop(['device'],1)

C:\Users\chexi\AppData\Local\Temp\ipykernel_2468\189213094.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df=df.drop(['device'],1)


In [12]:
df1=df.sort_values(['userId','sessionId','pageName','eventTime'])
df1['diff']=df1.groupby(['userId','sessionId','pageName'])['eventTime'].diff()
df2=df1[(df1['diff'].isna())]
df3=df1[(df1['diff'])>pd.Timedelta(seconds=5)]
df4=pd.concat([df2, df3], axis=0)
df4=df4.reset_index().sort_values(['index']).drop(['index','diff'],1).reset_index(drop=True)
print('去除',len(df)-len(df4),'条近似重复行')

去除 112 条近似重复行


C:\Users\chexi\AppData\Local\Temp\ipykernel_2468\1304559257.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df4=df4.reset_index().sort_values(['index']).drop(['index','diff'],1).reset_index(drop=True)


In [13]:
df4=df4.sort_values(['sessionId','eventTime']).reset_index(drop=True)

In [14]:
df4['prevPage']=df4.groupby('sessionId')['pageName'].shift(1).fillna('entry')

In [16]:
sessionSuccess=df4.groupby('sessionId')['pageName'].apply(lambda s: (s=='success').any()).rename('sessionSuccess')
df4=df4.merge(sessionSuccess, left_on='sessionId', right_index=True)

In [17]:
df4.to_csv('cleanedSimulatedData.csv',index=False)